In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import folium

def abrir_csv_con_indice(fichero_csv, columna_fecha, columnas_deseadas=None):

    def convertir_a_fecha(fecha):
        return pd.to_datetime(fecha, format='%Y-%m-%d')
    
    try:
        if columnas_deseadas:
            df = pd.read_csv(fichero_csv, parse_dates=[columna_fecha], date_parser=convertir_a_fecha, index_col=columna_fecha, usecols=columnas_deseadas, encoding='latin1')
        else:
            df = pd.read_csv(fichero_csv, parse_dates=[columna_fecha], date_parser=convertir_a_fecha, index_col=columna_fecha, encoding='latin1')
        
        df.sort_index(inplace=True)
        
        df.dropna(subset=['LAT_DECIMAL', 'LON_DECIMAL'], inplace=True)
        
        return df
    except MemoryError:
        print("El archivo es demasiado grande para leerlo de una vez. Considera leerlo por chunks.")
        return None


archivo_csv = "pacientesCovid.csv"
fecha_hosp = "FECHA_INGRESO"
columnas_deseadas = ["FECHA_INGRESO", "LAT_DECIMAL", "LON_DECIMAL"]
df_pacientesRespiratorios = abrir_csv_con_indice(archivo_csv, fecha_hosp, columnas_deseadas)


if df_pacientesRespiratorios is not None:
    print(df_pacientesRespiratorios.head())
    print(df_pacientesRespiratorios.tail())


/tmp/ipykernel_30336/1825246815.py:12: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(fichero_csv, parse_dates=[columna_fecha], date_parser=convertir_a_fecha, index_col=columna_fecha, usecols=columnas_deseadas, encoding='latin1')


               LAT_DECIMAL  LON_DECIMAL
FECHA_INGRESO                          
2020-01-01       19.482945   -99.113471
2020-01-01       19.359004   -99.092622
2020-01-01       19.482945   -99.113471
2020-01-01       19.390806   -99.195413
2020-01-01       19.288275   -99.167125
               LAT_DECIMAL  LON_DECIMAL
FECHA_INGRESO                          
2023-06-22       19.390806   -99.195413
2023-06-22       19.350214   -99.162146
2023-06-22       19.390806   -99.195413
2023-06-22       19.395901   -99.097612
2023-06-22       19.419261   -99.113701


In [2]:
kmeans = KMeans(n_clusters=5)

etiquetas = kmeans.fit_predict(df_pacientesRespiratorios)

df_pacientesRespiratorios['Cluster'] = etiquetas

centroides = kmeans.cluster_centers_

print(df_pacientesRespiratorios.head())

/home/rodrigo/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


               LAT_DECIMAL  LON_DECIMAL  Cluster
FECHA_INGRESO                                   
2020-01-01       19.482945   -99.113471        4
2020-01-01       19.359004   -99.092622        3
2020-01-01       19.482945   -99.113471        4
2020-01-01       19.390806   -99.195413        0
2020-01-01       19.288275   -99.167125        2


In [3]:
unique_points = df_pacientesRespiratorios.drop_duplicates(subset=['LAT_DECIMAL', 'LON_DECIMAL'])
unique_points_reset = unique_points.reset_index(drop=True)
unique_points_reset.to_csv('unique_points.csv', index=False)

print("Archivo 'unique_points.csv' guardado con éxito.")

Archivo 'unique_points.csv' guardado con éxito.


In [4]:
m = folium.Map(location=[df_pacientesRespiratorios['LAT_DECIMAL'].mean(), df_pacientesRespiratorios['LON_DECIMAL'].mean()], zoom_start=10)

coordenadas_usadas = set()  # Inicializa un conjunto vacío para almacenar las coordenadas usadas

for idx, (i, row) in enumerate(df_pacientesRespiratorios.iterrows()):
    coord_actual = (row['LAT_DECIMAL'], row['LON_DECIMAL'])
    
    if coord_actual not in coordenadas_usadas:
        coordenadas_usadas.add(coord_actual)  # Añade las coordenadas actuales al conjunto
        
        color = 'blue'  # color por defecto
        if etiquetas[idx] == 0:
            color = 'red'
        elif etiquetas[idx] == 1:
            color = 'green'
        elif etiquetas[idx] == 2:
            color = 'orange'
        elif etiquetas[idx] == 3:
            color = 'purple'
        elif etiquetas[idx] == 4:
            color = 'black'

        folium.CircleMarker(location=[row['LAT_DECIMAL'], row['LON_DECIMAL']], radius=5, color=color, fill=True, fill_color=color).add_to(m)

# Añadir centroides al mapa
for centroide in centroides:
    folium.Marker(location=[centroide[0], centroide[1]], icon=folium.Icon(color='gray')).add_to(m)

m.save("mapa_clusters.html")


In [5]:
df_pacientesRespiratorios.index = pd.to_datetime(df_pacientesRespiratorios.index)

conteo_semanal_por_cluster = df_pacientesRespiratorios.groupby('Cluster').resample('W').size().unstack(fill_value=0)

print(conteo_semanal_por_cluster.head())

FECHA_INGRESO  2020-01-05  2020-01-12  2020-01-19  2020-01-26  2020-02-02  \
Cluster                                                                     
0                      20          80          77          54          43   
1                       3          18          27          26          11   
2                      19          75          85          58          64   
3                      22          56          60          53          58   
4                      44         110         121          98          60   

FECHA_INGRESO  2020-02-09  2020-02-16  2020-02-23  2020-03-01  2020-03-08  \
Cluster                                                                     
0                      59          46          43          54          82   
1                      22          23          28          29          24   
2                      74          68          61          71          86   
3                      44          34          21          37          64  

In [6]:
conteo_semanal_por_cluster.to_csv('clusterSemanal.csv', index=True, date_format='%d-%m-%Y')
